In [ ]:
!pip install healpy

In [ ]:
import matplotlib.pyplot as plt
>>> import numpy as np
>>> import healpy as hp
>>> hp.mollview(np.arange(12))
>>> plt.show()

In [ ]:
import bz2
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib as mpl
from colossus.cosmology import cosmology  
from colossus.halo import mass_so
from colossus.lss import mass_function


In [ ]:
data = pd.read_csv("9305.csv.bz2", sep=",", comment='#', na_values=r'\N')

In [ ]:
unique_halo_id = data['unique_halo_id']
v_vir_cosmohub = data['v_vir']
z_cosmohub = data['z_cgal']
M_cosmohub = data['lmhalo']
ra_gal = data['ra_gal']
dec_gal = data['dec_gal']

In [ ]:
print(len(unique_halo_id),len(v_vir_cosmohub),len(z_cosmohub),len(M_cosmohub))

In [ ]:
print(len(set(unique_halo_id)))

In [ ]:
inds = []
unique_halo_id_sin_repetir = []
seen = set()
for i, halo in enumerate(unique_halo_id):
    if halo not in seen:
        inds.append(i)
        unique_halo_id_sin_repetir.append(halo)
    seen.add(halo)

In [ ]:
unique_halo_id = unique_halo_id_sin_repetir
v_vir_cosmohub = v_vir_cosmohub[inds]
z_cosmohub = z_cosmohub[inds]
M_cosmohub = M_cosmohub[inds]
ra_gal = ra_gal[inds]
dec_gal = dec_gal[inds]

In [ ]:
print(len(v_vir_cosmohub),len(z_cosmohub),len(M_cosmohub))

In [ ]:
v_vir_0_1_cosmohub = []
v_vir_1_2_cosmohub = []
M_0_1_cosmohub = []
M_1_2_cosmohub = []

for z,v_vir,M in zip(z_cosmohub,v_vir_cosmohub,M_cosmohub):
    if 1 > z >= 0:
        v_vir_0_1_cosmohub.append(v_vir)
        M_0_1_cosmohub.append(M)
    if 2 > z >= 1:
        v_vir_1_2_cosmohub.append(v_vir)
        M_1_2_cosmohub.append(M)
        
v_vir_0_1_cosmohub = np.array(v_vir_0_1_cosmohub)
v_vir_1_2_cosmohub = np.array(v_vir_1_2_cosmohub)
M_0_1_cosmohub = np.array(M_0_1_cosmohub)
M_1_2_cosmohub = np.array(M_1_2_cosmohub)

M_0_1_cosmohub = 10**M_0_1_cosmohub
M_1_2_cosmohub = 10**M_1_2_cosmohub
M_cosmohub = 10**M_cosmohub
M_cosmohub = np.array(M_cosmohub)

In [ ]:
def vc(M,Omega_m,Omega_lambda,h,z):
    Delta_c = mass_so.deltaVir(z)
    v_c=96.6*((Delta_c*Omega_m*(h**2)/24.4)**(1/6))*(((1+z)/3.3)**(1/2))*((M/10**11)**(1/3))
    return v_c

In [ ]:
params = cosmology.cosmologies['planck15']
params['Om0'] = 0.25
params['Ob0'] = 0.044
params['Ode0'] = 0.75
params['H0'] = 70
params['sigma8'] = 0.8
params['ns'] = 0.95
cosmo = cosmology.setCosmology('planck15',params)


# Parámetros de MICECAT (Best fit (WHISP))
Omega_m = 0.25
Omega_b = 0.044
Omega_lambda = 0.75
h = 0.7
alpha = 0.17
beta = -0.55
v_c0 = 37.1535  #km/s
v_c1 = 24547.08916 #km/s
M=np.logspace(10,15,100)  #En unidades solares

z = 0
v_c_0=vc(M_cosmohub,Omega_m,Omega_lambda,h,z)

z=1
v_c_1=vc(M_cosmohub,Omega_m,Omega_lambda,h,z)

z=2
v_c_2=vc(M_cosmohub,Omega_m,Omega_lambda,h,z)



plt.figure()
plt.loglog(M_cosmohub,v_c_0,label='teórica z=0')
plt.loglog(M_cosmohub,v_c_1,label='teórica z=1')
plt.loglog(M_cosmohub,v_c_2,label='teórica z=2')
plt.loglog(M_0_1_cosmohub,v_vir_0_1_cosmohub,label='cosmohub 1>z>0')
plt.loglog(M_1_2_cosmohub,v_vir_1_2_cosmohub,label='cosmohub 2>z>1')
plt.xlabel(r'$\frac{M}{M_{\odot}}$',fontsize=25)
plt.ylabel(r'$v_{virial}(M)$',fontsize=25)
plt.legend()
plt.show()

In [ ]:
def M_HI(M,alpha,beta,v_c0,v_c1,Omega_m,Omega_b,h,z,Delta_c,v_c):
    #v_c=96.6*((Delta_c*Omega_m*(h**2)/24.4)**(1/6))*(((1+z)/3.3)**(1/2))*((M/10**11)**(1/3))
    f=(Omega_b/Omega_m)*(0.75) # (1-Y)*Omega_b/Omega_m
    return alpha*f*M*((M*h/1e11)**beta)*np.exp(-(v_c0/v_c)**3)*np.exp(-(v_c/v_c1)**3)

In [ ]:
mpl.rcParams['agg.path.chunksize'] = 10000
M = np.logspace(10,15,1000)

plt.figure()
plt.loglog(M_cosmohub,M_HI(M_cosmohub,alpha,beta,v_c0,v_c1,Omega_m,Omega_b,h,z_cosmohub,mass_so.deltaVir(z_cosmohub),v_vir_cosmohub),'k.',label=r'M cosmohub $v_{vir}$ cosmohub')
plt.loglog(M_cosmohub,M_HI(M_cosmohub,alpha,beta,v_c0,v_c1,Omega_m,Omega_b,h,z_cosmohub,mass_so.deltaVir(z_cosmohub),vc(M_cosmohub,Omega_m,Omega_lambda,h,z_cosmohub)),'r.',label=r'M cosmohub $v_{vir}$ teórica (z cosmohub)')
plt.loglog(M,M_HI(M,alpha,beta,v_c0,v_c1,Omega_m,Omega_b,h,1,mass_so.deltaVir(1),vc(M,Omega_m,Omega_lambda,h,1)),'b-',label=r'M teórica $v_{vir}$ teórica (z=1)')
plt.legend()
plt.xlabel(r'$\frac{M}{M_{\odot}}$',fontsize=25)
plt.ylabel(r'$\frac{M_{HI}}{M_{\odot}}$',fontsize=25)
plt.show()

In [ ]:
M_0_8_0_9_cosmohub = []
z_0_8_0_9_cosmohub = []
ra_gal_0_8_0_9 = []
dec_gal_0_8_0_9 = []
v_vir_cosmohub_0_8_0_9 = []

for z,M,ra,dec,v_vir in zip(z_cosmohub,M_cosmohub,ra_gal,dec_gal,v_vir_cosmohub):
    if 0.9 >= z >= 0.8:
        M_0_8_0_9_cosmohub.append(M)
        z_0_8_0_9_cosmohub.append(z)
        ra_gal_0_8_0_9.append(ra)
        dec_gal_0_8_0_9.append(dec)
        v_vir_cosmohub_0_8_0_9.append(v_vir)

M_0_8_0_9_cosmohub = np.array(M_0_8_0_9_cosmohub)
z_0_8_0_9_cosmohub = np.array(z_0_8_0_9_cosmohub)
ra_gal_0_8_0_9 = np.array(ra_gal_0_8_0_9)
dec_gal_0_8_0_9 = np.array(dec_gal_0_8_0_9)
v_vir_cosmohub_0_8_0_9 = np.array(v_vir_cosmohub_0_8_0_9)

M_intervalos = np.logspace(np.log10(min(M_0_8_0_9_cosmohub)),np.log10(max(M_0_8_0_9_cosmohub)),500)
numbers = []

for i in range(len(M_intervalos)-1):
    indexes = np.where(np.logical_and(M_0_8_0_9_cosmohub>=M_intervalos[i], (M_0_8_0_9_cosmohub<=M_intervalos[i+1])))[0]
    number = len(indexes)
    numbers.append(number)

M_intervalos = np.delete(M_intervalos,-1)
numbers = np.array(numbers)

In [ ]:
def n(Masa, z):
    n = mass_function.massFunction(Masa, z, mdef = 'fof', model = 'crocce10', q_out = 'dndlnM')
    return n

In [ ]:
z = 0.85
Masa = np.logspace(11,15)
potencia = np.log10(Masa[1]/Masa[0])
dMasa = Masa*(10**potencia-1)
dMasa = np.array(dMasa)
A = sum(n(Masa, z)*dMasa)
A

In [ ]:
def n(Masa, z):
    n = (1/A)*mass_function.massFunction(Masa, z, mdef = 'fof', model = 'crocce10', q_out = 'dndlnM')
    return n

In [ ]:
mhist,edges_mass = np.histogram(M_0_8_0_9_cosmohub,bins=100000)
Mass_array = (edges_mass[:-1] + edges_mass[1:])/2
dMasa = edges_mass[1:] - edges_mass[:-1]
B = sum(mhist*dMasa)
mhist = mhist/B

plt.figure()
plt.plot(Mass_array,mhist,label='halos MICE 0.9 > z > 0.8')
plt.plot(Mass_array,n(Mass_array,0.8),label=r'n(M,z) ${(h/Mpc)}^3$ con z=0.80')
plt.plot(Mass_array,n(Mass_array,0.85),label=r'n(M,z) ${(h/Mpc)}^3$ con z=0.85')
plt.plot(Mass_array,n(Mass_array,0.9),label=r'n(M,z) ${(h/Mpc)}^3$ con z=0.90')
plt.xlabel(r'$\frac{M}{M_{\odot}}$',fontsize=25)
plt.ylabel('Cada uno con sus unidades')
plt.xscale('log')
plt.yscale('log')
plt.title('Usando histogram')
plt.legend()
plt.show()


In [ ]:
#En Mpc/h
coords_x = data['xhalo']
coords_y = data['yhalo']
coords_z =data['zhalo']

lado_x = max(coords_x)-min(coords_x)
lado_y = max(coords_y)-min(coords_y)
lado_z = max(coords_z)-min(coords_z)

In [ ]:
print('Lado x Lado y Lado z')
print(lado_x,lado_y,lado_z,' (Mpc/h)')

In [ ]:
#En (Mpc/h)**3
fSky = 1/8
Vol = (4/3)*np.pi*(cosmo.comovingDistance(0,0.9,True)**3 - cosmo.comovingDistance(0,0.8,True)**3)*fSky
Vol

In [ ]:
#Adimensional
N_halos = n(Masa,z)*Vol


z = 0.85

plt.figure()
plt.plot(Mass_array,mhist,label='halos MICE 0.9 > z > 0.8')
plt.plot(Masa,N_halos,label=r'halos (n(M,z)Vol) con z=0.85')
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$\frac{M}{M_{\odot}}$',fontsize=25)
plt.title('Usando histogram')
plt.legend()
plt.show()


In [ ]:
#Adimensional



z = 0.85

plt.figure()
plt.plot(Mass_array,mhist/Vol,label='halos MICE 0.9 > z > 0.8')
plt.plot(Masa,n(Masa,z),label=r'halos (n(M,z)Vol) con z=0.85')
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$\frac{M}{M_{\odot}}$',fontsize=25)
plt.title('Usando histogram')
plt.legend()
plt.show()


In [ ]:
densidad_critica = cosmo.rho_c(0.7)*1e9/h
print('Densidad crítica ',densidad_critica,'(7.85 10^11) Msol h^3 /Mpc^3')

In [ ]:
print(max(coords_x),min(coords_x),'Mpc/h')
print(max(coords_y),min(coords_y),'Mpc/h')
print(max(coords_z),min(coords_z),'Mpc/h')

Creamos un cubo de lado 3060 Mpc/h (por redondear) y lo dividimos en 128x128x128 celdas.

In [ ]:
Cubo_x = np.linspace(0,3060,129)  #101 fronteras para que haya 100 celdas
Cubo_y = np.linspace(0,3060,129)  #101 fronteras para que haya 100 celdas
Cubo_z = np.linspace(0,3060,129)  #101 fronteras para que haya 100 celdas

Vcelda = (Cubo_x[1] - Cubo_x[0])**3
print(Vcelda,'(Mpc/h)^3')

In [ ]:
from bisect import bisect_left

In [ ]:
#Se clasifican en celdas de manera que el límite superior de la celda cuenta 
#como la celda anterior (excepto el primer límite de la primera celda,
#que cuenta como la primera celda)

Densidad_3D_oscura = np.zeros((128,128,128))
posiciones_x = []
posiciones_y = []
posiciones_z = []

for x,y,z,m in zip(coords_x,coords_y,coords_z,M_cosmohub):
    pos_x = bisect_left(Cubo_x, x)  #np.digitize
    pos_x = pos_x - 1
    if pos_x == -1:
        pos_x = 0
    pos_y = bisect_left(Cubo_y, y)
    pos_y = pos_y - 1
    if pos_y == -1:
        pos_y = 0
    pos_z = bisect_left(Cubo_z, z)
    pos_z = pos_z - 1
    if pos_z == -1:
        pos_z = 0
    posiciones_x.append(pos_x)
    posiciones_y.append(pos_y)
    posiciones_z.append(pos_z)
    Densidad_3D_oscura[pos_x,pos_y,pos_z] = Densidad_3D_oscura[pos_x,pos_y,pos_z] + (m/Vcelda)
    
#Primer indice controla la matriz en la que está, el segundo la fila
#y el tercero la columna de esa matriz.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib

In [ ]:
plt.figure()
fig_imshow = plt.imshow(Densidad_3D_oscura[:,:,50])
plt.xlabel('Lado x')
plt.ylabel('Lado y')
plt.title('Materia Oscura')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\rho_{Oscura} \; (M_{\odot}h^3/{Mpc}^3)$',fontsize=16)
plt.show()


In [ ]:
plt.figure()
fig_imshow = plt.imshow((1/densidad_critica)*Densidad_3D_oscura[:,:,50])
plt.xlabel('Lado x')
plt.ylabel('Lado y')
plt.title('Materia Oscura')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\delta_c$',fontsize=16)
plt.show()


In [ ]:
plt.figure()
fig_imshow = plt.imshow(Densidad_3D_oscura[30:70,30:70,50])
plt.xlabel('Lado x')
plt.ylabel('Lado y')
plt.title('Materia Oscura')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\rho_{Oscura} \; (M_{\odot}h^3/{Mpc}^3)$',fontsize=16)
plt.show()


In [ ]:
plt.figure()
fig_imshow = plt.imshow((1/densidad_critica)*Densidad_3D_oscura[30:70,30:70,50])
plt.xlabel('Lado x')
plt.ylabel('Lado y')
plt.title('Materia Oscura')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\delta_c$',fontsize=16)
plt.show()


In [ ]:
#fig = plt.figure()
#ax = fig.gca(projection='3d')
#ax.scatter(posiciones_x,posiciones_y,posiciones_z,s=0.001,c=M_cosmohub/Vcelda)
#ax.set_xlabel('Lado x')
#ax.set_ylabel('Lado y')
#ax.set_zlabel('Lado z')
#ax.set_title('Materia Oscura')
#plt.show()

In [ ]:
print(max(dec_gal_0_8_0_9),min(dec_gal_0_8_0_9),'grados')
print(max(ra_gal_0_8_0_9),min(ra_gal_0_8_0_9),'grados')

In [ ]:
print(max(dec_gal),min(dec_gal),'grados')
print(max(ra_gal),min(ra_gal),'grados')

Cortar en 0 a 90 para ambas

Curioso que la ra solo vaya entre 0 y 90 para las galaxias de estos redshift. Cogiendo todo el catálogo, iban entre -270 y 90. Cortar entre 0 y 90.

In [ ]:
lado_ra = np.linspace(0,90,129)  #129 fronteras para que haya 128 celdas
lado_dec = (180/np.pi)*np.arccos(np.linspace(np.cos(0),np.cos(np.pi/2),129))
lado_dec_lin = np.linspace(0,90,129)
lado_dec_izq = lado_dec[:-1]
lado_dec_der = lado_dec[1:]
lado_dec_lin_izq = lado_dec_lin[:-1]
lado_dec_lin_der = lado_dec_lin[1:]


delta_ra = lado_ra[1] - lado_ra[0]
delta_dec = lado_dec[1] - lado_dec[0]

Vcelda_esferica = (((np.cos(lado_dec_izq*2*np.pi/360)-np.cos(lado_dec_der*2*np.pi/360))*delta_ra*2*np.pi/360)/(4*np.pi))*(4/3)*np.pi*(cosmo.comovingDistance(0,0.9,transverse=False)**3-cosmo.comovingDistance(0,0.8,transverse=False)**3)
Vcelda_esferica_lin = (((np.cos(lado_dec_lin_izq*2*np.pi/360)-np.cos(lado_dec_lin_der*2*np.pi/360))*delta_ra*2*np.pi/360)/(4*np.pi))*(4/3)*np.pi*(cosmo.comovingDistance(0,0.9,transverse=False)**3-cosmo.comovingDistance(0,0.8,transverse=False)**3)
print(Vcelda_esferica,'(Mpc/h)^3')


$V_{celda} = f_{Sky} \; \frac{4}{3} \pi \left({\chi(z=0.9)}^3 - {\chi(z=0.8)}^3 \right)$

$f_{Sky} = \frac{\int_{0}^{\Delta \theta} \int_{0}^{\Delta \phi}{sen \theta \; d\theta \; d\phi}}{\int_{0}^{\pi} \int_{0}^{2\pi} {sen \theta \; d\theta \; d\phi}} = \frac{\left(1 - cos \Delta\theta \right) \; \Delta \phi }{4\pi}$

In [ ]:
#Esta vez se van a ir guardando según sus coordenadas angulares

#Se clasifican en celdas de manera que el límite superior de la celda cuenta 
#como la celda anterior (excepto el primer límite de la primera celda,
#que cuenta como la primera celda).

Densidad_0_8_0_9_2D_oscura = np.zeros((128,128))
angulos_dec = []
angulos_ra = []

for dec,ra,m in zip(dec_gal_0_8_0_9,ra_gal_0_8_0_9,M_0_8_0_9_cosmohub):
    ang_dec = bisect_left(lado_dec_lin, dec)  #np.digitize
    ang_dec = ang_dec - 1
    if ang_dec == -1:
        ang_dec = 0
    ang_ra = bisect_left(lado_ra, ra)
    ang_ra = ang_ra - 1
    if ang_ra == -1:
        ang_ra = 0
    angulos_dec.append(ang_dec)
    angulos_ra.append(ang_ra)
    Densidad_0_8_0_9_2D_oscura[ang_dec,ang_ra] = Densidad_0_8_0_9_2D_oscura[ang_dec,ang_ra] + m
    
#Primer indice controla la fila
#y el segundo la columna de esa matriz.

In [ ]:
plt.figure()
fig_imshow = plt.imshow(Densidad_0_8_0_9_2D_oscura)
plt.xlabel('Ra')
plt.ylabel('Dec')
plt.title('Materia Oscura 0.8 < z < 0.9')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\rho_{Oscura} \; (M_{\odot}h^3/{Mpc}^3)$',fontsize=16)
plt.show()

In [ ]:
plt.figure()
fig_imshow = plt.imshow((1/densidad_critica)*Densidad_0_8_0_9_2D_oscura)
plt.xlabel('Ra')
plt.ylabel('Dec')
plt.title('Materia Oscura 0.8 < z < 0.9')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\delta_c$',fontsize=16)
plt.show()

In [ ]:
lado_ra_sample = np.linspace(35,40,100)  #129 fronteras para que haya 128 celdas
lado_dec_sample = np.linspace(35,40,100)


delta_ra_sample = lado_ra_sample[1] - lado_ra_sample[0]
delta_dec = lado_dec[1] - lado_dec[0]

Vcelda_esferica_sample = (((1-np.cos(delta_dec*2*np.pi/360))*delta_ra*2*np.pi/360)/(4*np.pi))*(4/3)*np.pi*(cosmo.comovingDistance(0,0.9,transverse=False)**3-cosmo.comovingDistance(0,0.8,transverse=False)**3)
print(Vcelda_esferica,'(Mpc/h)^3')

In [ ]:
#Esta vez se van a ir guardando según sus coordenadas angulares

#Se clasifican en celdas de manera que el límite superior de la celda cuenta 
#como la celda anterior (excepto el primer límite de la primera celda,
#que cuenta como la primera celda).

dec_gal_sample = dec_gal_0_8_0_9[np.where((ra_gal_0_8_0_9>35)&(ra_gal_0_8_0_9<40)&(dec_gal_0_8_0_9>35)&(dec_gal_0_8_0_9<40))[0]]
ra_gal_sample = ra_gal_0_8_0_9[np.where((ra_gal_0_8_0_9>35)&(ra_gal_0_8_0_9<40)&(dec_gal_0_8_0_9>35)&(dec_gal_0_8_0_9<40))[0]]
M_sample = M_0_8_0_9_cosmohub[np.where((ra_gal_0_8_0_9>35)&(ra_gal_0_8_0_9<40)&(dec_gal_0_8_0_9>35)&(dec_gal_0_8_0_9<40))[0]]

Densidad_sample_2D_oscura = np.zeros((100,100))

angulos_dec = []
angulos_ra = []

for dec,ra,m in zip(dec_gal_sample,ra_gal_sample,M_sample):
    ang_dec = bisect_left(lado_dec, dec)  #np.digitize
    ang_dec = ang_dec - 1
    if ang_dec == -1:
        ang_dec = 0
    ang_ra = bisect_left(lado_ra, ra)
    ang_ra = ang_ra - 1
    if ang_ra == -1:
        ang_ra = 0
    angulos_dec.append(ang_dec)
    angulos_ra.append(ang_ra)
    Densidad_sample_2D_oscura[ang_dec,ang_ra] = Densidad_sample_2D_oscura[ang_dec,ang_ra] + (m/Vcelda_esferica_sample)
    
#Primer indice controla la fila
#y el segundo la columna de esa matriz.

In [ ]:
plt.figure()
fig_imshow = plt.imshow(Densidad_sample_2D_oscura)
plt.xlabel('Ra')
plt.ylabel('Dec')
plt.title('Materia Oscura (0.8 < z < 0.9) (ra y dec 35 < 40)')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\rho_{Oscura} \; (M_{\odot}h^3/{Mpc}^3)$',fontsize=16)
plt.show()

Poner los ángulos en los ejes

In [ ]:
A12 = 2.876e-15 ##Hz
h_planck =6.62607004e-34 #m2 kg / s
nu21 = 1420e6 #Hz
m_h = 1.673723e-27 #kg
kboltz = 1.38064852e-23 #m2 kg s-2 K-1
c_light_meter = 3.0e8 #ms-1

In [ ]:
#Se clasifican en celdas de manera que el límite superior de la celda cuenta 
#como la celda anterior (excepto el primer límite de la primera celda,
#que cuenta como la primera celda)

Densidad_3D_HI = np.zeros((128,128,128))
T_3D = np.zeros((128,128,128))

posiciones_x = []
posiciones_y = []
posiciones_z = []

for x,y,z,m,Z in zip(coords_x,coords_y,coords_z,M_HI(M_cosmohub,alpha,beta,v_c0,v_c1,Omega_m,Omega_b,h,z,mass_so.deltaVir(z_cosmohub),v_vir_cosmohub),z_cosmohub):
    pos_x = bisect_left(Cubo_x, x)  #np.digitize
    pos_x = pos_x - 1
    if pos_x == -1:
        pos_x = 0
    pos_y = bisect_left(Cubo_y, y)
    pos_y = pos_y - 1
    if pos_y == -1:
        pos_y = 0
    pos_z = bisect_left(Cubo_z, z)
    pos_z = pos_z - 1
    if pos_z == -1:
        pos_z = 0
    posiciones_x.append(pos_x)
    posiciones_y.append(pos_y)
    posiciones_z.append(pos_z)
    Densidad_3D_HI[pos_x,pos_y,pos_z] = Densidad_3D_HI[pos_x,pos_y,pos_z] + (m/Vcelda)
    T_3D[pos_x,pos_y,pos_z] = T_3D[pos_x,pos_y,pos_z] + (3*h_planck*c_light_meter**3*A12)/(32*np.pi*kboltz*m_h*nu21**2)*((1+Z)**2)/(cosmo.Hz(Z))*(m/Vcelda)
    
#Primer indice controla la matriz en la que está, el segundo la fila
#y el tercero la columna de esa matriz.

In [ ]:
plt.figure()
plt.imshow(Densidad_3D_HI[:,:,50])
plt.xlabel('Lado x')
plt.ylabel('Lado y')
plt.title('HI')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\rho_{HI} \; (M_{\odot}h^3/{Mpc}^3)$',fontsize=16)
plt.show()

Ese valor tan bajo no coincide con los valores que veo de la matriz. Parece que falta el exponente de 10?

Divido la densidad de HI también entre la crítica o mejor eso solo para los halos?

In [ ]:
plt.figure()
plt.imshow((1/densidad_critica)*Densidad_3D_HI[:,:,50])
plt.xlabel('Lado x')
plt.ylabel('Lado y')
plt.title('HI')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\delta_c$',fontsize=16)
plt.show()

Como es que la sobredensidad da igual que la densidad?

In [ ]:
plt.figure()
plt.imshow(Densidad_3D_HI[30:70,30:70,50])
plt.xlabel('Lado x')
plt.ylabel('Lado y')
plt.title('HI')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\rho_{HI} \; (M_{\odot}h^3/{Mpc}^3)$',fontsize=16)
plt.show()

In [ ]:
plt.figure()
plt.imshow((1/densidad_critica)*Densidad_3D_HI[30:70,30:70,50])
plt.xlabel('Lado x')
plt.ylabel('Lado y')
plt.title('HI')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\delta_c$',fontsize=16)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(posiciones_x,posiciones_y,posiciones_z,s=0.001,c=M_HI(M_cosmohub,alpha,beta,v_c0,v_c1,Omega_m,Omega_b,h,z,mass_so.deltaVir(z_cosmohub),v_vir_cosmohub)/Vcelda)
ax.set_xlabel('Lado x')
ax.set_ylabel('Lado y')
ax.set_zlabel('Lado z')
ax.set_title('HI')
plt.show()

In [ ]:
plt.figure()
plt.imshow(T_3D[:,:,50])
plt.xlabel('Lado x')
plt.ylabel('Lado y')
plt.title('Temperatura HI')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label('T(K)',fontsize=16)
plt.show()

In [ ]:
#Ahora igual pero con las coordenadas angulares

#Se clasifican en celdas de manera que el límite superior de la celda cuenta 
#como la celda anterior (excepto el primer límite de la primera celda,
#que cuenta como la primera celda)

Densidad_0_8_0_9_2D_HI = np.zeros((128,128))
T_0_8_0_9_2D = np.zeros((128,128))
angulos_dec = []
angulos_ra = []

for ra,dec,m,Z in zip(ra_gal_0_8_0_9,dec_gal_0_8_0_9,M_HI(M_0_8_0_9_cosmohub,alpha,beta,v_c0,v_c1,Omega_m,Omega_b,h,z,mass_so.deltaVir(z_0_8_0_9_cosmohub),v_vir_cosmohub_0_8_0_9),z_0_8_0_9_cosmohub):
    ang_dec = bisect_left(lado_dec, dec)  #np.digitize
    ang_dec = ang_dec - 1
    if ang_dec == -1:
        ang_dec = 0
    ang_ra = bisect_left(lado_ra, ra)
    ang_ra = ang_ra - 1
    if ang_ra == -1:
        ang_ra = 0
    angulos_dec.append(ang_dec)
    angulos_ra.append(ang_ra)
    
    Densidad_0_8_0_9_2D_HI[ang_dec,ang_ra] = Densidad_0_8_0_9_2D_HI[ang_dec,ang_ra] + (m/Vcelda_esferica)
    T_0_8_0_9_2D[ang_dec,ang_ra] = T_0_8_0_9_2D[ang_dec,ang_ra] + (3*h_planck*c_light_meter**3*A12)/(32*np.pi*kboltz*m_h*nu21**2)*((1+Z)**2)/(cosmo.Hz(Z))*(m/Vcelda_esferica)
    
#Primer indice controla la fila y el segundo la columna de la matriz.

In [ ]:
plt.figure()
fig_imshow = plt.imshow(Densidad_0_8_0_9_2D_HI)
plt.xlabel('Ra')
plt.ylabel('Dec')
plt.title('HI 0.8 < z < 0.9')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\rho_{HI} \; (M_{\odot}h^3/{Mpc}^3)$',fontsize=16)
plt.show()

In [ ]:
plt.figure()
fig_imshow = plt.imshow((1/densidad_critica)*Densidad_0_8_0_9_2D_HI)
plt.xlabel('Ra')
plt.ylabel('Dec')
plt.title('HI 0.8 < z < 0.9')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label(r'$\delta_c$',fontsize=16)
plt.show()

In [ ]:
plt.figure()
plt.imshow(T_0_8_0_9_2D)
plt.xlabel('Ra')
plt.ylabel('Dec')
plt.title('Temperatura HI 0.8 < z < 0.9')
cbar = plt.colorbar(fig_imshow, extend='both')
cbar.set_label('T(K)',fontsize=16)
plt.show()

In [ ]:
data

In [ ]:
z = 0.85

plt.figure()
plt.plot(M_intervalos,numbers/sum(numbers),label='halos MICE 0.9 > z > 0.8')
plt.plot(Masa,n(Masa,z)/sum(n(Masa,z)),label=r'n(M,z) ${(h/Mpc)}^3 con z=0.85$')
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$\frac{M}{M_{\odot}}$',fontsize=25)
plt.ylabel('Normalizado')
plt.title('Usando intervalos')
plt.legend()
plt.show()

In [ ]:
#Adimensional
N_halos = n(Masa,z)*Vol


z = 0.85

plt.figure()
plt.plot(M_intervalos,numbers,label='halos MICE 0.9 > z > 0.8')
plt.plot(Masa,N_halos,label=r'halos (n(M,z)Vol) con z=0.85')
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$\frac{M}{M_{\odot}}$',fontsize=25)
plt.title('Usando intervalos')
plt.legend()
plt.show()